In [1]:
import pandas as pd
import numpy as np
from biopsy_grade_extractor import BiopsyGradeExtractor # load exttractor

### load the dataset which contrains biopsy reports

In [2]:
df_pathology = pd.read_csv('../data/labeled_PSA.csv')	

In [3]:
def get_report_number(report):
    return report.split('accession number: ')[1].split(' report')[0]

df_pathology['Report_Number'] = df_pathology['Report_Text'].apply(get_report_number)

In [4]:
# df_pathology_test = pd.read_csv('../data/data.csv')

In [5]:
# df_pathology_test = df_pathology_test[df_pathology_test.Report_Number.isin(['S17-61498','MS08G21552'])]

### Instantiate BiopsyGradeExtractor and fit the data

In [6]:
# extractor = BiopsyGradeExtractor()
# df_result_test = extractor.fit(df_pathology_test, display_summary = True, return_summary_df = True)

In [49]:
extractor = BiopsyGradeExtractor()
df_result= extractor.fit(df_pathology, display_summary = True, return_summary_df = True)

extracing info...: 100%|██████████| 10118/10118 [00:04<00:00, 2524.56it/s]



Extraction complete.


Displaying summary...
Counts per each primary grade: 
3.0    7869
4.0    3176
5.0     213
2.0     130
3.5      26
4.5       6
2.5       4
1.0       3
0.0       1
dtype: int64


Counts per each secondary grade: 
4.0    5207
3.0    5034
5.0     989
2.0     155
3.5      26
4.5       6
2.5       4
1.0       4
6.0       2
7.0       1
dtype: int64


Number of reports per each overall grade : 
2    3913
1    2682
3    1572
5     710
4     520
dtype: int64
total =  9397
Number of unique biopsy reports :  10118
Number of unique patients :  10118
Number of unique patients with overall_grade_merged :  9397



/Users/guangyawan/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
/Users/guangyawan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [16]:
df_result['Report_Text'][df_result['overall_grade_merged'].isna()].sample(100).to_excel('../data/missing_report.xlsx')

In [10]:
df_result[['Report_Text','overall_grade_merged']]['overall_grade_merged'].value_counts()

2    2781
1    1335
3    1194
5     434
4     416
Name: overall_grade_merged, dtype: int64

In [21]:
df_result.overall_grade_merged.iloc[0]

In [10]:
df_pathology.iloc[1]['Report_Text']

NameError: name 'df_pathology' is not defined

### Validate the performance of the algorithm

In [5]:
# if report_number_oi is not specified, then it randomly picks report to evaluate
for i in range(2):
    extractor.validate_results()



Extracted info : 
Report_Number                                                  SPNPSGOSBB
Report_Text             LEFT (A3): Benign prostate tissue. LEFT MID (B...
primary_grade                                                         [3]
secondary_grade                                                       [3]
overall_grade_merged                                                  1.0
Name: 17, dtype: object


Pathology report : 
LEFT (A3): Benign prostate tissue. LEFT MID (B3): Benign prostate tissue. LEFT APEX (C3): Benign prostate tissue. RIGHT BASE (D3): Benign prostate tissue. RIGHT MID (E3): PROSTATIC ADENOCARCINOMA, Gleason grade 3+3, involving 1 of 1 core, approximately 15% of tissue. RIGHT APEX (F3): Benign prostate tissue.




Extracted info : 
Report_Number                                                  SPNPSGOSBB
Report_Text             LEFT (A3): Benign prostate tissue. LEFT MID (B...
primary_grade                                                         [3]
secondary_g

In [ ]:
# Primary Grade/Secondary Grade/overall grade merged

In [37]:
df_result.columns

Index(['Unnamed: 0', 'EMPI', 'MRN', 'hospital_rp', 'date_rp_dt',
       'date_path_dt', 'path_postRP_days', 'PLND', 'LNM', 'time_lab_result',
       'PSA', 'PSAdate_minus_RPdate', 'Report_Text', 'Report_Text_noAN',
       'Report_Number', 'primary_grade', 'secondary_grade',
       'overall_grade_merged'],
      dtype='object')

In [50]:
df_result = df_result.drop(columns = df_result.columns[[0,2,3,4,5,6,9,11,12,13,14]])

In [51]:
df_MRI = pd.read_csv('../data/MRI.csv')

In [52]:
df_MRI = df_MRI.drop(columns= ['Report_Number','Report_Text'],axis = 1)

In [53]:
df_result = df_result.merge(df_MRI,on='EMPI',how='left')

In [54]:
df_result = df_result.drop(columns = ['primary_grade','secondary_grade','PLND'],axis = 1)

In [64]:
print(df_result.shape)
for col in df_result.columns[1:]:
    if(col == 'binned'):
        continue
    print(f'Number of Missing data for column {col} : ', df_result [col].isna().sum())
    if(df_result[col].value_counts().shape[0] > 10):
        bins = df_result[col].quantile([0,.25, .5,.75,1])
        df_result['binned'] = pd.cut(df_result[col], bins)
        print (df_result['binned'].value_counts())
        continue
    print(df_result[col].value_counts())

(10766, 7)
Number of Missing data for column LNM :  0
0    10497
1      269
Name: LNM, dtype: int64
Number of Missing data for column PSA :  5429
(4.34, 6.07]     1339
(9.29, 177.5]    1334
(6.07, 9.29]     1329
(0.009, 4.34]    1309
Name: binned, dtype: int64
Number of Missing data for column overall_grade_merged :  877
2    4127
1    2743
3    1693
5     780
4     546
Name: overall_grade_merged, dtype: int64
Number of Missing data for column lymphadenopathy :  7335
False    2084
True     1347
Name: lymphadenopathy, dtype: int64
Number of Missing data for column pirads :  10063
5.0    336
4.0    312
3.0     51
2.0      3
1.0      1
Name: pirads, dtype: int64
Number of Missing data for column binned :  5455
(4.34, 6.07]     1339
(9.29, 177.5]    1334
(6.07, 9.29]     1329
(0.009, 4.34]    1309
Name: binned, dtype: int64


In [63]:
df_result['binned'].value_counts()

(4.34, 6.07]     1339
(9.29, 177.5]    1334
(6.07, 9.29]     1329
Name: binned, dtype: int64

In [56]:
df_result.columns

Index(['EMPI', 'LNM', 'PSA', 'overall_grade_merged', 'lymphadenopathy',
       'pirads'],
      dtype='object')